<a href="https://colab.research.google.com/github/mariacmartins/bioinformatica-disciplina/blob/main/Aula_6_Aprendizado_de_M%C3%A1quina_%E2%80%93_Aprendizado_N%C3%A3o_Supervisionado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 6: Aprendizado de Máquina – Aprendizado Não-Supervisionado

O Aprendizado de Máquina (Machine Learning) é uma sub-área da Inteligência Artificial dedicada à replicação do processo de aprendizagem. Os algoritmos de ML aprendem com base em dados. 

O aprendizado de máquina supervisionado utiliza dados rotulados para desenvolvimento do modelo. Já **o aprendizado não-supervisionado** caracteriza-se pelo aprendizado com **base em dados não-rotulados**, permitindo a identificação de similaridades, agrupamentos (*clusters*), anomalias e novas formas de representar um determinado conjunto de dados. 

**Exemplo de aplicação em bioinformática (aprendizado não-supervisionado):** Representação visual e identificação de linhagens celulares em dados de *single-cell sequencing.* 

### PhageProtDetector:
Agora daremos continuidade ao projeto de análise de proteínas de bactérias e fagos iniciado na aula passada. Imagine que você está analisando as características das proteínas destes dois grupos de organismos e gostaria de verificar a disposição dos dados no espaço de features é suficiente para se realizar a classificação. 

- Carregue os dados para um DataFrame 
- Crie as variáveis X, y, contendo, respectivamente, os dados de composição de aminoácido e os rótulos das proteínas baixadas (campo “phage”).

In [1]:
import pandas as pd
import plotly.express as px

In [2]:
df = pd.read_csv('http://bit.ly/bioinfo-ml-phage-dataset')
df.head()

,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y,phage
0,0.034857,0.004648,0.062742,0.054996,0.055771,0.046476,0.007746,0.099148,0.065066,0.072812,0.018590,0.116189,0.034082,0.026336,0.039504,0.075910,0.061967,0.054222,0.010844,0.058095,1.0
1,0.028213,0.007053,0.063480,0.056426,0.050940,0.040752,0.009404,0.097179,0.074451,0.085423,0.014890,0.098746,0.036834,0.033699,0.030564,0.090125,0.057210,0.057994,0.013323,0.053292,1.0
2,0.027972,0.010490,0.055944,0.024476,0.024476,0.013986,0.006993,0.080420,0.055944,0.094406,0.013986,0.153846,0.010490,0.076923,0.031469,0.097902,0.066434,0.052448,0.034965,0.066434,1.0
3,0.053968,0.015873,0.085714,0.047619,0.028571,0.069841,0.025397,0.073016,0.057143,0.076190,0.028571,0.057143,0.041270,0.031746,0.041270,0.047619,0.066667,0.126984,0.003175,0.022222,1.0
4,0.061856,0.006873,0.082474,0.082474,0.054983,0.061856,0.024055,0.072165,0.072165,0.096220,0.020619,0.041237,0.020619,0.030928,0.054983,0.054983,0.054983,0.054983,0.010309,0.041237,1.0


In [3]:
X = df.drop(['phage'], axis=1)
y = df['phage']

-	Utilize o algoritmo TSNE para gerar 2 “componentes” a partir das composições de aminoácidos (X), e armazene esta informação em um novo DataFrame junto do rótulo original.

In [4]:
from sklearn.manifold import TSNE

In [5]:
tsne = TSNE(n_components=2)
components = tsne.fit_transform(X)

df_tsne = pd.DataFrame(components, columns=['tsne_components_1', 'tsne_components_2'])
df_tsne['phage'] = y.astype(str)
df_tsne.head()

,tsne_components_1,tsne_components_2,phage
0,52.905399,0.338295,1.0
1,53.017857,0.254335,1.0
2,51.587139,-1.064247,1.0
3,16.743950,29.075558,1.0
4,32.448456,-54.013512,1.0


- Crie um gráfico com o plotly para mostrar os componentes produzidos e o rótulo original.

In [6]:
fig = px.scatter(df_tsne, x='tsne_components_1', y='tsne_components_2', color='phage')
fig.show()

- Utilize o algoritmo DBSCAN para gerar clusters a partir dos componentes produzidos pelo TSNE.
- Avalie os clusters produzidos utilizando as métricas homogeneidade (homogeneity_score) e completude (completeness_score).
- Crie um gráfico com o plotly para mostrar os componentes produzidos e os clusters identificados pelo DBSCAN. 

In [7]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import homogeneity_score, completeness_score

In [14]:
dbscan = DBSCAN(eps=1.5)
df_tsne['dbscan_cluster'] = dbscan.fit_predict(df_tsne[['tsne_components_1', 'tsne_components_2']]).astype(str)

In [15]:
fig = px.scatter(df_tsne, x='tsne_components_1', y='tsne_components_2', color='dbscan_cluster')
fig.show()

In [16]:
homogeneity_score(y, df_tsne['dbscan_cluster'])

0.6459977361622333

In [17]:
completeness_score(y, df_tsne['dbscan_cluster'])

0.08888131242094967